!pip install git+https://github.com/moroots/TOLNET_DEMO

In [1]:
from TOLNET_DEMO.tolnet import TOLNet

In [2]:
tolnet = TOLNet()

In [3]:
#Prints querible parameters.
tolnet.print_product_types()
# tolnet.print_processing_types()
# tolnet.print_instrument_groups()
# tolnet.print_file_types()


 TOLNET product IDs:
 id product_type_name
  1           O3Lidar
  2           Surface
  3             Other
  4             HIRES
  5            CALVAL
  6              CLIM
  7           Gridded
  8            Legacy 



In [ ]:
date_start = "2023-08-08"
date_end = "2023-08-11"
product_IDs = [4]

# Fetches data using the TOLNet object. min_date and max_date are required, but other parameters are optional.
# Stores the data in the object, which can be accessed with tolnet.data and tolnet.meta_data.
# This will prompt the user for confirmation if the query would download every file from the API.
# Querible parameters are processing_type, instrument_group, product_type, and file_type. 
# All parameters require a list of IDs.
data = tolnet.import_data(min_date=date_start, max_date=date_end, product_type=product_IDs)


 44%|████████████████████████████████████▎                                              | 7/16 [00:14<00:17,  1.89s/it]

In [ ]:
# Plots the data, used after import_data. Puts each combination of instrument group and processing type on a seperate plot.
# Additional parameters can be used: title, ylabel, xlabel(axis labels), xlims, ylims, yticks, surface, sonde, and savefig.
# Title, ylabel, and xlabel take strings.
# xlims takes a list of two dates in ISO 8601 Format(YYYY-MM-DD), like this: ['2023-08-05', '2023-08-08'].
# Surface and Sonde plot their respective datas instead.
data.tolnet_curtains()